# Ejercicio: Analizar texto

* Utilizar un texto de [proyecto Gutenberg en castellano](http://www.gutenberg.org/browse/languages/es) 
* Contar palabras y ordenar por frecuencia:
    * Limpiar preludio y licencia de Project Gutenberg
    * Omitir “palabras vacías” (stop words) y símbolos
* Encontrar personajes
* Hacer un análisis extra a gusto
* Hint: Tutorial para español: [PLN Python](https://relopezbriega.github.io/blog/2017/09/23/procesamiento-del-lenguaje-natural-con-python/)
    * Usa [spacy.io](https://spacy.io/) y [github.com/chartbeat-labs/textacy](https://github.com/chartbeat-labs/textacy)
    * No hace falta la parte de Deep Learning

In [1]:
import spacy
import textacy
from collections import defaultdict, Counter

Ejecutar la siguiente sentencia para **descargar el modulo español de Spacy**:

                        python -m spacy download es

---

Vamos a trabajar con el cuento "**Ratón Pérez: cuento infantil**" de Coloma, Luis, 1851-1915

In [2]:
TEXT_FILE = '../input/raton_perez.txt'

Encoding del cuento:

In [3]:
import chardet

with open(TEXT_FILE, 'rb') as f:
    result = chardet.detect(f.read())

result

{'encoding': 'UTF-8-SIG', 'confidence': 1.0, 'language': ''}

Eliminamos del texto, las siguientes cosas:
* Salto de linea: \n
* Espacios vacios: ''
* Inserciones de imagenes: \[images\]
* Astericos: '\*       \*       \*       \*       \*'

In [4]:
clean_text = []
with open(TEXT_FILE, 'r', encoding='UTF-8-SIG') as f:
    asterisks = '*       *       *       *       *'
    for line in f.readlines():
        line = line.strip()  # Eliminamos los \n
        
        # Eliminamos:
        #     - Espacios vacios
        #     - "imagenes" ([imagen])
        #     - astericos (*     *     *)
        if (line != '' and
            not line.startswith("[imagen") and
            line != asterisks):
            clean_text.append(line)

Limpiamos la **Licencia del Proyecto Gutenberg** y el **Preludio**.

Para saber cual es el preludio, nos fijamos donde comienza y termina el cuento.  
Para ellos nos guiamos de la version PDF del cuento: http://www.textos.info/luis-coloma/raton-perez/pdf

In [5]:
# Clean Prelude and Gutenberg license
begin_story = 'Sembrad en los niños la idea, aunque no la entiendan: los años se'
end_story = 'mayor_, acá en la tierra...'

raton_perez = clean_text[clean_text.index(begin_story):
                         clean_text.index(end_story) + 1]

# Pasamos el texto a lower case
#raton_perez = list(map(str.lower, raton_perez))

In [6]:
print("# Lineas del cuento: {}".format(len(raton_perez)))

# Lineas del cuento: 361


In [7]:
raton_perez_text = " ".join(raton_perez)

raton_perez_text[:1000]

'Sembrad en los niños la idea, aunque no la entiendan: los años se encargarán de descifrarla en su entendimiento y hacerla florecer en su corazón. Entre la muerte del rey que rabió y el advenimiento al trono de la reina Mari-Castaña existe un largo y obscuro período en las crónicas, de que quedan pocas memorias. Consta, sin embargo, que floreció en aquella época un rey Buby I, grande amigo de los niños pobres y protector decidido de los ratones. Fundó una fábrica de muñecos y caballos de cartón para los primeros, y sábese de cierto, que de esta fábrica procedían los tres caballitos cuatralbos, que regaló el rey D. Bermudo _el Diácono_ á los niños de Hissén I, después de la batalla de Bureva. Consta también que el rey Buby prohibió severamente el uso de ratoneras y dictó muy discretas leyes para encerrar en los límites de la defensa propia los instintos cazadores de los gatos: lo cual resulta probado, por los graves disturbios que hubo entre la reina doña Goto ó Gotona, viuda de D. Sanc

Procesemos el cuento:

In [8]:
# Cargamos el modulo español de Spacy
nlp = spacy.load('es')

raton_perez_doc = nlp(raton_perez_text)

In [9]:
# Sentencias que hay en el texto
rp_sents = list(raton_perez_doc.sents)

print("# Sentences = {}".format(len(rp_sents)))

# Sentences = 123


In [10]:
rp_sents[:10]

[Sembrad en los niños la idea, aunque no la entiendan: los años se encargarán de descifrarla en su entendimiento y hacerla florecer en su corazón.,
 Entre la muerte del rey que rabió y el advenimiento al trono de la reina Mari-Castaña existe un largo y obscuro período en las crónicas, de que quedan pocas memorias.,
 Consta, sin embargo, que floreció en aquella época un rey Buby I, grande amigo de los niños pobres y protector decidido de los ratones.,
 Fundó una fábrica de muñecos y caballos de cartón para los primeros, y sábese de cierto, que de esta fábrica procedían los tres caballitos cuatralbos, que regaló el rey D. Bermudo _el Diácono_ á los niños de Hissén I, después de la batalla de Bureva.,
 Consta también que el rey Buby prohibió severamente el uso de ratoneras y dictó muy discretas leyes para encerrar en los límites de la defensa propia los instintos cazadores de los gatos: lo cual resulta probado, por los graves disturbios que hubo entre la reina doña Goto ó Gotona, viuda de

In [11]:
type(raton_perez_doc)

spacy.tokens.doc.Doc

Eliminamos las Stopwords y los signos de puntuacion

In [12]:
rp_words_list = []
for token in raton_perez_doc:
    if not token.is_stop and not token.is_punct:
        token_text = token.text
        rp_words_list.append(token_text.lower())

10 palabras mas frecuentes, con su respectivo conteo:

In [13]:
Counter(rp_words_list).most_common(10)

[('y', 174),
 ('á', 82),
 ('rey', 45),
 ('buby', 44),
 ('ratón', 34),
 ('pérez', 33),
 ('diente', 12),
 ('reina', 9),
 ('oro', 9),
 ('niños', 8)]

Entidades que aparecen en el cuento, con su respectiva cantidad de apariciones:

In [14]:
Counter(ent.label_ for ent in raton_perez_doc.ents)

Counter({'PER': 120, 'MISC': 76, 'LOC': 70, 'ORG': 6})

Cojunto de Personas o Familias nombradas:

In [15]:
ents_PER = set(ent.text for ent in raton_perez_doc.ents if ent.label_ == 'PER')

print("# PER Entities = {}\n".format(len(ents_PER)))

print(ents_PER)

# PER Entities = 63

{'Atáronle', 'Ratón Pérez', 'Parecía', 'D. Gaiferos', 'rey Buby', 'Sirvieron', 'Recogiólo', 'Había', 'Doblaron', 'Cheese', 'Cogió', 'la Reina', 'Ana Stuard', 'Buby', 'S. M. la Reina', 'Adolfo', 'Apretóle la Reina', 'Abel', 'Carlos Prats', 'Miss Old', 'D. Lucas de Tuy', 'Así', 'D. Sancho Ordóñez', 'Mari', 'la Virgen', 'Allí', 'Detrás', 'Gilito', 'Amanecía', 'Confesó', 'Sembrad', 'Castaña', 'Querían', 'Consta', 'Antojósele', 'Acostóse', 'Acordóse', 'Su Majestad', 'Hissén I', 'Vaseo', 'Alarmóse', 'Grande', 'Hizo', 'Sévres', 'Entró', 'Elvira', 'Á', 'Caín', 'Gilitos', 'Reina', 'Carlos Prats[A', 'Acercóse Ratón Pérez', 'Agradó', 'rey D. Bermudo', 'J. Tate', 'Pero S. M.', 'Puso', 'Quedóse', 'Dios', 'Oíale', 'Huntley', 'Rey', '¡Pero'}


Entidades diversas, por ejemplo:
* Eventos
* Nacionalidades
* Productos
* Obras de arte.

In [16]:
ents_MISC = set(ent.text for ent in raton_perez_doc.ents if ent.label_ == 'MISC')

print("# MISC Entities = {}\n".format(len(ents_MISC)))

print(ents_MISC)

# MISC Entities = 63

{'Ratón Pérez', 'Entraba por las innumerables rendijas el viento helado del alba', 'Le está encomendando el alma', 'Señora', 'Minini', 'rey Buby', 'Jockey-Club', 'Los médicos quisieron cloroformizarle', 'Era su carrera desatinada', 'Todo era entrada en aquella miserable habitación abierta á todos los vientos', 'El miedo', 'Fuera de sí de contento', 'Toisón de Oro', 'batalla de Ecnoma', 'Tennis_.', 'El rey Buby', 'De pronto dijo', 'Con la rapidez', 'Charcot', 'Tan sólo', 'Padre nuestro', 'Corona', 'Presidente del Consejo', 'Pero el rey Buby', 'Miróle', 'Quiso', 'Incorporóse', 'Reyecito', 'La Reina', 'La guardia ratonil', 'De pronto, sintió', 'Agradecióselo mucho el', 'Mas Ratón Pérez', 'Las señoritas hacían labor con su aya Miss Old-', 'Niño Jesús de Praga', 'Ripers', 'Vivía Ratón Pérez', 'Al calorcito de la lumbre oculta bajo el rescoldo dormía', 'De repente vió el rey Buby', 'Era entonces el rey Buby', 'La vanidad venció á Ratón Pérez', 'S. M. la hora de mudar lo

Nombre de la ubicación definida política o geográficamente, por ejemplo:
* Ciudades
* Provincias
* Países
* Regiones internacionales
* Cuerpos de agua
* Montañas

In [17]:
ents_LOC = set(ent.text for ent in raton_perez_doc.ents if ent.label_ == 'LOC')

print("# LOC Entities = {}\n".format(len(ents_LOC)))

print(ents_LOC)

# LOC Entities = 47

{'Pues', 'No', 'Creyó', 'Adelaida', '¡Ni mantas quería él ya tener en su cama', 'Monasterio de Pombeyro', 'Caminaba', 'Galicia', 'Comprendió', 'Buby', 'Reyecito', 'Consejo de Ministros', 'Desdémona', 'Goto ó Gotona', 'Gilito', 'Reunióse', 'Holanda', 'Cesó', 'Gaiferos', 'Consta', 'Habló', 'Conocíase', 'Pérez', 'Convocó', 'Termópilas', 'Pareceres', 'Facultad', 'Apuradillo', 'Reina', 'Encaramóse', 'Polo', 'Fundó', 'Dejólo', 'Diácono', 'Reino', 'Gruyère', 'Cómo', 'Hospital del Rey', 'Mas', 'Ratona Pérez', 'Despertó', 'Tornó', 'Monarca', 'Ministro de Estado', 'calle del Arenal', 'Cámara', 'Cazadores'}


Nombre corporativo, gubernamental u otra entidad organizacional:

In [18]:
ents_ORG = set(ent.text for ent in raton_perez_doc.ents if ent.label_ == 'ORG')

print("# ORG Entities = {}\n".format(len(ents_ORG)))

print(ents_ORG)

# ORG Entities = 6

{'Cronicón Iriense', 'Oliva[B', 'Contestóle Ratón Pérez', 'Club', 'Embajada alemana', 'Merindad'}


Algunas estadisticas del cuento, usando Textacy:

In [19]:
rp_stats = textacy.text_stats.TextStats(raton_perez_doc)

In [20]:
print("* # Oraciones = {}".format(rp_stats.n_sents))
print("* # Palabras = {}".format(rp_stats.n_words))
print("* # Palabras unicas = {}".format(rp_stats.n_unique_words))
print("* # Palabras de largo mayor o igual 7 = {}".format(rp_stats.n_long_words))
print("* # Caracteres = {}".format(rp_stats.n_chars))
print("* # Silabas = {}".format(rp_stats.n_syllables))

* # Oraciones = 123
* # Palabras = 3869
* # Palabras unicas = 1381
* # Palabras de largo mayor o igual 7 = 875
* # Caracteres = 17285
* # Silabas = 7045
